# This is a comparison of the original code and after revision

Install requried modules

In [1]:
%pip install networkx
%pip install matplotlib
%pip install tqdm
%pip install numpy
%pip install ipywidgets
%pip install pandas
%pip install scikit-learn
%pip install Jinja2
%pip install pyyaml

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install torch

In [3]:
%pip install ray

In [4]:
%pip install "dask[distributed]"

Note: you may need to restart the kernel to use updated packages.


Imports

In [5]:
import os
import time
import random
import pickle
import networkx as nx
import subprocess
import matplotlib.pyplot as plt
import numpy as np
import hashlib
import yaml
import json
import math

from pathlib import Path
from shutil import rmtree
from subprocess import CalledProcessError, TimeoutExpired
from multiprocessing import Lock
from os import makedirs, replace, cpu_count, getpid
from os.path import join, exists
from tqdm.notebook import tqdm

from itertools import permutations

from concurrent.futures import ProcessPoolExecutor, as_completed

In [6]:
# from scripts.original.main import do_operations as ORIGINAL_SCRIPT

In [7]:
# from scripts.paralized.main import do_operations as PARALLEL_SCRIPT

Configuration

In [8]:
class Dataset:
    def __init__(self, dir, classes):
        self.dir = join("scripts", dir)
        self.classes = classes

    def __str__(self):
        return self.dir

In [9]:
DATASETS = [
    Dataset(
        dir="ten_newsgroups_graphs",
        classes=['business', 'entertainment', 'food', 'graphics', 'historical', 'medical', 'politics', 'space', 'sport', 'technologie']
    ),

    Dataset(
        dir="bbcsport_graphs",
        classes=['athletics', 'cricket', 'football', 'rugby', 'tennis']
    )
]

for dataset in DATASETS:
    makedirs(dataset.dir, exist_ok=True)

#SAMPLE_SIZES = [1, 5, 10, 50, 100, 500, 1000, 2500, 5000, 10000, 25000, 50000]
SAMPLE_SIZES = [10, 50, 100, 500, 1000, 5000]

NODE_TYPES = ['article', 'author', 'organization', 'topic']
EDGE_TYPES = ['cites_by_test1', 'mentions_to_test2', 'authored_by_test3', 'belongs_to_test4', 'related_to_test5']

NOTE_DISTRIBUTION = {
    'article': 0.6,
    'author': 0.2,
    'organization': 0.1,
    'topic': 0.1
}

EDGE_DISTRIBUTION = [
    0.3,
    0.25,
    0.2,
    0.15,
    0.1
]

TRAIN_TEST_RATIO = 0.7
MIN_NODES = 5
MAX_NODES = 30

NUM_WORKERS = cpu_count() // 2
LOCK = Lock()

Functions to generate test data

In [10]:
BASE_DIR = os.getcwd()

In [11]:
def join(*args):
    """Универсальное объединение путей через pathlib"""
    return Path(*args)

In [12]:
def makedirs(path, exist_ok=False):
    """Создает директорию с родительскими, если необходимо"""
    path = Path(path)
    path.mkdir(parents=True, exist_ok=exist_ok)
    return path

In [13]:
def init_paths(datasets):
    """Создает всю структуру директорий заранее"""
    required_paths = [
        join(BASE_DIR, dataset.dir, class_name)
        for dataset in datasets
        for class_name in dataset.classes
    ]
    
    for path in required_paths:
        makedirs(path, exist_ok=True)
        if not path.exists():
            raise RuntimeError(f"Failed to create directory: {path}")

In [14]:
def safe_save_labels(class_dir, class_name, node_types, edge_types):
    """Потокобезопасное сохранение меток"""
    class_path = Path(class_dir)
    
    # Исправлено создание словаря для vertex_labels
    vertex_labels = {node_type: idx for idx, node_type in enumerate(node_types)}
    vertex_path = join(class_path, f"{class_name}_vertex_labels.pickle")
    with vertex_path.open('wb') as f:
        pickle.dump(vertex_labels, f)
    
    # Исправлено создание словаря для edge_labels
    edge_labels = {tuple(et.split('_')): idx for idx, et in enumerate(edge_types)}
    edge_path = join(class_path, f"{class_name}_edge_labels.pickle")
    with edge_path.open('wb') as f:
        pickle.dump(edge_labels, f)

In [15]:
def generate_and_save_patterns():
    """Генерирует и сохраняет концепты и частые подграфы с проверкой наличия vertex_labels"""
    for dataset in DATASETS:
        for class_name in dataset.classes:
            class_dir = Path(BASE_DIR) / dataset.dir / class_name
            class_dir.mkdir(exist_ok=True)

            safe_save_labels(class_dir, class_name, NODE_TYPES, EDGE_TYPES)

            # Проверка наличия vertex_labels
            vertex_path = join(class_dir, f"{class_name}_vertex_labels.pickle")
            if not vertex_path.exists():
                raise FileNotFoundError(
                    f"Vertex labels missing for {class_name}! "
                    f"Expected at: {vertex_path}"
                )

            # Загрузка меток
            with vertex_path.open('rb') as f:
                vertex_labels = pickle.load(f)

            edge_path = join(class_dir, f"{class_name}_edge_labels.pickle")
            with open(edge_path, 'rb') as f:
                edge_labels = pickle.load(f)

            # Генерация 40 концептов для каждого класса
            concepts = []
            for i in range(40):
                concept = {
                    'subgraphs': [
                        f"{random.choice(['node', 'edge'])}{j}-{random.randint(0, 100)}"
                        for j in range(random.randint(2, 5))
                    ],
                    'supports': [
                        round(random.uniform(0.1, 1.0), 2)
                        for _ in range(random.randint(2, 4))
                    ],
                    'extent': [
                        str(random.randint(100, 1000))
                        for _ in range(random.randint(3, 6))
                    ],
                    'id': f"{class_name}_concept_{i}"
                }
                concepts.append(concept)

            # Генерация частых подграфов
            # Генерация частых подграфов (с защитой от пустых структур)
            frequent_subgraphs = []
            for i in range(30):
                G = nx.MultiDiGraph()
                
                # Гарантируем минимум 2 узла и 1 ребро
                num_nodes = random.randint(2, 6)
                num_edges = random.randint(1, num_nodes * (num_nodes - 1))
                
                # Добавляем узлы с гарантированными метками
                valid_labels = list(vertex_labels.keys())
                if not valid_labels:
                    raise ValueError(f"No vertex labels for {class_name}")
                
                for node_id in range(num_nodes):
                    G.add_node(node_id, label=random.choice(valid_labels))
                
                # Добавляем ребра с гарантированными парами узлов
                all_pairs = list(permutations(range(num_nodes), 2))
                edges_to_add = random.sample(all_pairs, min(num_edges, len(all_pairs)))
                
                for u, v in edges_to_add:
                    edge_type = random.choice(list(edge_labels.keys()))
                    G.add_edge(u, v, label=edge_type)
                
                # Пропускаем графы без ребер (защита)
                if len(G.edges()) == 0:
                    continue
                
                frequent_subgraphs.append({
                    'id': f"{class_name}_frequent_subgraph_{i}",
                    'subgraphs': [G],
                    'extent': [str(random.randint(1, 100)) for _ in range(5, 12)]
                })

            # Генерация паттернов
            patterns = []
            vertex_labels_list = list(vertex_labels.values())
            edge_labels_list = list(edge_labels.values())

            for pattern_id in range(40):
                num_nodes = random.randint(1, 5)
                nodes = [
                    (node_id, random.choice(vertex_labels_list))
                    for node_id in range(num_nodes)
                ]

                edges = []
                if num_nodes > 1:
                    num_edges = random.randint(0, min(4, num_nodes * 2))
                    for _ in range(num_edges):
                        u, v = random.sample(range(num_nodes), 2)
                        edge_type = random.choice(edge_labels_list)
                        edges.append((u, v, edge_type))

                graph_ids = random.sample(range(100), random.randint(10, 20))

                pattern = [
                    f"# {random.randint(10, 100)}",
                    f"t # {pattern_id}",
                    *[f"v {node_id} {label}" for node_id, label in nodes],
                    *[f"e {u} {v} {label}" for u, v, label in edges],
                    f"x {' '.join(map(str, graph_ids))}"
                ]
                patterns.extend(pattern)

            # Сохранение файлов внутри цикла по классам
            concepts_path = join(class_dir, f'{class_name}_concepts.pickle')
            with open(concepts_path, 'wb') as f:
                pickle.dump(concepts, f)

            frequent_subgraphs_path = join(class_dir, f'{class_name}_frequent_subgraphs.pickle')
            with open(frequent_subgraphs_path, 'wb') as f:
                pickle.dump(frequent_subgraphs, f)

            # Исправлено: сохранение в текстовом формате вместо pickle
            patterns_out_path = join(class_dir, f'{class_name}_patterns.OUT')
            with open(patterns_out_path, 'w') as f:
                f.write('\n'.join(patterns))  # Записываем строки как текст

In [16]:
import json
import random
from pathlib import Path

def generate_lattice_file(class_dir, class_name):
    """Генерирует и сохраняет файл lattice для указанного класса."""
    lattice_structure = [
        {
            "NodesCount": random.randint(50, 150),
            "ArcsCount": random.randint(100, 300),
            "Top": [random.randint(0, 50)],
            "Bottom": random.sample(range(0, 150), 25),
            "Params": {
                "Type": "ContextProcessorModules",
                "Name": "SofiaContextProcessorModule",
                "Params": {
                    "Thld": 5.0,
                    "MaxPatternNumber": 10000,
                    "AdjustThreshold": True,
                    "FindPartialOrder": True,
                    "OutputParams": {"OutExtent": True, "OutIntent": True},
                    "ProjectionChain": {
                        "Type": "ProjectionChainModules",
                        "Name": "StabClsPatternProjectionChainModule",
                        "Params": {
                            "ImageReserve": 31000,
                            "Enumerator": {
                                "Type": "PatternEnumeratorModules",
                                "Name": "ParallelPatternEnumeratorModule",
                                "Params": {
                                    "PatternEnumeratorByCallback": {
                                        "Type": "PatternEnumeratorModules",
                                        "Name": "GastonGraphPatternEnumeratorModule",
                                        "Params": {
                                            "LibPath": "libGaston.so",
                                            "InputPath": f"../../../../LibGastonForSofia/sample-data/{class_name}_train_gsofia",
                                            "RemoveInput": False,
                                            "PatternPath": f"{class_name}_patterns.OUT",
                                            "MinSupport": 18,
                                            "MaxGraphSize": -1,
                                            "GastonMode": "Full"
                                        }
                                    }
                                }
                            }
                        }
                    }
                }
            }
        },
        {
            "Nodes": [
                {
                    "Ext": {
                        "Count": random.randint(20, 100),
                        "Inds": random.sample(range(100), random.randint(15, 30))
                    },
                    "Int": {
                        "GraphCount": random.randint(10, 50),
                        "MinGraphSupport": random.randint(20, 100)
                    },
                    "Interest": random.randint(5, 15)
                } for _ in range(40)
            ],
            "Arcs": [
                {
                    "S": random.randint(0, 100),
                    "D": random.randint(0, 100)
                } for _ in range(200)
            ]
        }
    ]
    
    lattice_path = Path(class_dir) / f"{class_name}_lattice.json"
    with lattice_path.open('w', encoding='utf-8') as f:
        json.dump(lattice_structure, f, indent=2, ensure_ascii=False)


In [17]:
def safe_file_operations(path, content):
    """Безопасная запись файлов с проверкой контрольных сумм"""
    temp_path = f"{path}.tmp"
    with open(temp_path, 'wb') as f:
        pickle.dump(content, f)
    
    with open(temp_path, 'rb') as f:
        checksum = hashlib.md5(f.read()).hexdigest()
    
    Path(temp_path).replace(path)
    return checksum

In [18]:
def generate_dynamic_probs(n):
    """Генерирует нормализованные убывающие вероятности для заданного количества элементов"""
    if n <= 0:
        return np.array([])
    base = np.exp(-0.5 * np.arange(n))
    base /= base.sum()
    return base

In [19]:
def init_worker():
    """Инициализация генератора случайных чисел для каждого процесса"""
    np.random.seed((getpid() * int(time.time())) % 123456789)

In [20]:
def generate_graph(args, num_graphs):
    init_paths(DATASETS)
    
    class_dir, class_name, node_types, node_probs, edge_probs, i = args
    G = nx.DiGraph()
    prefix = 'train' if i < num_graphs * TRAIN_TEST_RATIO else 'test'

    num_nodes = random.randint(MIN_NODES, MAX_NODES)
    if num_nodes == 0:
        num_nodes = 1  # Гарантируем хотя бы одну вершину

    for node_id in range(num_nodes):
        node_type = np.random.choice(node_types, p=node_probs)
        content = f"{node_type.capitalize()} about {class_name} - {np.random.randint(1000, 9999)}"
        G.add_node(
            node_id,
            label=node_type,
            content=content,
            class_label=class_name,
            creation_date=f"2023-{random.randint(1,12):02d}-{random.randint(1,28):02d}"
        )

    all_nodes = list(G.nodes())
    if len(all_nodes) > 1:
        for u in all_nodes:
            if random.random() < 0.7:
                targets = [n for n in all_nodes if G.nodes[n]['class_label'] == class_name]
            else:
                targets = all_nodes
            
            if targets and u in targets:
                targets.remove(u)  # Исключаем саму вершину
                n_targets = len(targets)
                
                if n_targets > 0:
                    probs = generate_dynamic_probs(n_targets)
                    try:
                        selected = np.random.choice(
                            targets,
                            size=min(3, n_targets),
                            replace=False,
                            p=probs
                        )
                        for v in selected:
                            edge_type = np.random.choice(EDGE_TYPES, p=edge_probs)
                            weight = round(np.random.beta(2, 5), 2)
                            G.add_edge(
                                u, v,
                                label=edge_type,
                                weight=weight,
                                timestamp=np.random.randint(1609459200, 1672531200)
                            )
                    except ValueError:
                        pass

    for node in G.nodes(data=True):
        node[1]['content'] = str(node[1]['content'])
        node[1]['creation_date'] = str(node[1]['creation_date'])

    graph_path = join(class_dir, f'{class_name}_{prefix}_graph_{i}.gml')
    with LOCK:
        nx.write_gml(G, graph_path)

In [21]:
def delete_data_dirs():
    for dataset in DATASETS:
        for class_name in dataset.classes:
            class_dir = join(BASE_DIR, dataset.dir, class_name)
            if class_dir.exists():
                rmtree(class_dir)

In [22]:
def parallel_generate_gSOFIA_data(num_graphs):
    """Генерация графов и структуры данных с настраиваемым количеством графов"""
    assert abs(sum(NOTE_DISTRIBUTION.values()) - 1.0) < 1e-6, "Node distribution error"
    assert abs(sum(EDGE_DISTRIBUTION) - 1.0) < 1e-6, "Edge distribution error"

    node_types = list(NOTE_DISTRIBUTION.keys())
    node_probs = list(NOTE_DISTRIBUTION.values())
    edge_probs = list(EDGE_DISTRIBUTION)

    total_classes = sum([len(dataset.classes) for dataset in DATASETS])
    print("Total classes:", total_classes)
    total_graphs_per_class =  int(round(num_graphs * (1 + TRAIN_TEST_RATIO), 0)) // total_classes

    tasks = []
    for dataset in DATASETS:
        futures = []
        for class_name in dataset.classes:
            class_dir = join(BASE_DIR, dataset.dir, class_name)
            makedirs(class_dir, exist_ok=True)
            
            safe_save_labels(class_dir, class_name, node_types, EDGE_TYPES)
            
            # Генерация графов (train + test)
            
            
            for i in range(total_graphs_per_class):
                # prefix = 'train' if i < num_graphs * TRAIN_TEST_RATIO else 'test'
                # graph_path = join(class_dir, f'{class_name}_{prefix}_graph_{i}.gml')
                
                # Генерация и сохранение графа
                futures.append(
                    generate_graph((str(class_dir), class_name, node_types, node_probs, edge_probs, i), total_graphs_per_class)
                )

            generate_lattice_file(str(class_dir), class_name)

            total_classes += 1

        with ProcessPoolExecutor(max_workers=NUM_WORKERS, initializer=init_worker) as executor:
            # futures = [executor.submit(generate_graph, task, num_graphs=num_graphs // len(dataset.classes)) for task in tasks]
            for _ in tqdm(futures, total=len(futures), desc="Generating graphs"):
                pass

    generate_and_save_patterns()
    print(f"Generated data for {total_graphs_per_class * total_classes} graphs")

Function to run scripts

In [23]:
def run_script(script_func):
    """Запуск с правильными параметрами для вашей системы"""

    root = os.getcwd()
    start_time = time.time()

    config = join(root, 'config', 'config.yaml')
    with open(config) as file:
        config = yaml.load(file, Loader=yaml.FullLoader)

    dataset = "all"
    mode = "all"
    weighting = "no"
    graph_pattern_reconstruction = "no"

    try:
        script_func(root=root,
                    config=config,
                    dataset=dataset,
                    mode=mode,
                    weighting=weighting,
                    graph_pattern_reconstruction=graph_pattern_reconstruction)
            
    except CalledProcessError as e:
        print(f"Process error in {script_func}:\n{e.stderr}")
        return float('inf')

    except TimeoutExpired:
        print(f"Timeout in {script_func}")
        return float('inf')
    
    return time.time() - start_time

Data generation

In [24]:
# Генерация тестовых данных
# print("Generating test data...")
# for size in tqdm(SAMPLE_SIZES):
#     parallel_generate_gSOFIA_data(size)

In [25]:
delete_data_dirs()
parallel_generate_gSOFIA_data(10)

Total classes: 15


Generating graphs:   0%|          | 0/10 [00:00<?, ?it/s]

Generating graphs:   0%|          | 0/5 [00:00<?, ?it/s]

Generated data for 30 graphs


Benchmarking

In [21]:
# Замер производительности
original_times = []
parallel_times = []

print("Benchmarking...")
for size in tqdm(SAMPLE_SIZES):

    # Удаляем старые данные
    delete_data_dirs()

    # Генерируем новые данные
    parallel_generate_gSOFIA_data(size)
    
    # Оригинальная версия
    original_times.append(
        run_script(ORIGINAL_SCRIPT)
    )
    
    # Параллельная версия
    parallel_times.append(
        run_script(PARALLEL_SCRIPT)
    )

Benchmarking...


  0%|          | 0/6 [00:00<?, ?it/s]

Generating graphs:   0%|          | 0/110 [00:00<?, ?it/s]

Scoring for ten_newsgroups for concepts done.
{'dataset': 'ten_newsgroups', 'mode': 'concepts', 'baseline_penalty_macro-averaged_f1-score': 0.6424603339706314, 'penalty_1_macro-averaged_f1-score': 0.6932400698716489, 'penalty_2_macro-averaged_f1-score': 0.6765980400897618, 'penalty_3_macro-averaged_f1-score': 0.6453613579403052, 'edge_penalty_macro-averaged_f1-score': 0.5259470797401831, 'penalty_5_macro-averaged_f1-score': 0.667410319470691, 'penalty_6_macro-averaged_f1-score': 0.7114699627857523}


c:\project\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\project\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\project\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

c:\project\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\project\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\project\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Scoring for bbcsport for equivalence classes done.
{'dataset': 'bbcsport', 'mode': 'equivalence_classes', 'baseline_penalty_macro-averaged_f1-score': 0.7413663243075008, 'penalty_1_macro-averaged_f1-score': 0.9227173113624726, 'penalty_2_macro-averaged_f1-score': 0.9045502645502645, 'penalty_3_macro-averaged_f1-score': 0.8941479484875711, 'edge_penalty_macro-averaged_f1-score': 0.5437306949241847, 'penalty_5_macro-averaged_f1-score': 0.9084054834054834, 'penalty_6_macro-averaged_f1-score': 0.9299900386351998}
Scoring for bbcsport for frequent subgraphs done.
{'dataset': 'bbcsport', 'mode': 'frequent_subgraphs', 'baseline_penalty_macro-averaged_f1-score': 0.7012962962962963, 'penalty_1_macro-averaged_f1-score': 0.9299900386351998, 'penalty_2_macro-averaged_f1-score': 0.9299900386351998, 'penalty_3_macro-averaged_f1-score': 0.9186996993448606, 'edge_penalty_macro-averaged_f1-score': 0.9436090225563909, 'penalty_5_macro-averaged_f1-score': 0.8639122315592903, 'penalty_6_macro-averaged_f1-

(score_graph_patterns pid=4328) Scoring graph patterns for ten_newsgroups...
(score_graph_patterns pid=13536) Scoring graph patterns for bbcsport...
(raylet) WARNING: 40 PYTHON worker processes have been started on node: 9776bd256faade10808b6b64054a21f34947db306d27fad668b32bf8 with address: 127.0.0.1. This could be a result of using a large number of actors, or due to tasks blocked in ray.get() calls (see https://github.com/ray-project/ray/issues/3644 for some discussion of workarounds).
(raylet) WARNING: 50 PYTHON worker processes have been started on node: 9776bd256faade10808b6b64054a21f34947db306d27fad668b32bf8 with address: 127.0.0.1. This could be a result of using a large number of actors, or due to tasks blocked in ray.get() calls (see https://github.com/ray-project/ray/issues/3644 for some discussion of workarounds).
(raylet) WARNING: 60 PYTHON worker processes have been started on node: 9776bd256faade10808b6b64054a21f34947db306d27fad668b32bf8 with address: 127.0.0.1. This coul

(raylet) [2025-02-27 19:25:04,124 C 4908 13616] dlmalloc.cc:129:  Check failed: *handle != nullptr CreateFileMapping() failed. GetLastError() = 1450
(raylet) *** StackTrace Information ***
(raylet) unknown
(raylet) 


LocalRayletDiedError: The task's local raylet died. Check raylet.out for more information.

Results plot

In [ ]:
# Визуализация результатов
plt.figure(figsize=(12, 6))
plt.plot(SAMPLE_SIZES, original_times, marker='o', label='Original')
plt.plot(SAMPLE_SIZES, parallel_times, marker='x', label='Parallel')
plt.xlabel('Number of Graphs')
plt.ylabel('Execution Time (seconds)')
plt.title('Performance Comparison')
plt.legend()
plt.grid(True)
plt.yscale('log')
plt.xscale('log')
plt.show()

Mean results

In [ ]:
# Расчет ускорения
speedup = np.array(original_times) / np.array(parallel_times)
print(f"Average speedup: {np.mean(speedup):.2f}x")
print(f"Max speedup: {np.max(speedup):.2f}x")